<a href="https://colab.research.google.com/github/adimehta98/UFC-Betting-Strategy/blob/main/artGAN_128.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
random_seed = 1
from numpy.random import seed
seed(random_seed)
import random
random.seed(random_seed)
import pandas as pd
import numpy as np 
import os 
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as Vutil
from torch.autograd import Variable

In [ ]:
reloadModels = True
nIter= 1000
imageSize = (128,128)
batchSize = 128

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc. 
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

Running on the GPU


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
imageFolder = "/content/drive/MyDrive/Pytorch DCGAN art /Sample/pics/" 
resultsFolder = "/content/drive/MyDrive/Pytorch DCGAN art /Results/"
modelFolder = "/content/drive/MyDrive/Pytorch DCGAN art /Models/"
transform = transforms.Compose([transforms.Scale(imageSize),transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
artData = dset.ImageFolder(imageFolder,transform=transform)
dataloader = torch.utils.data.DataLoader(artData,batch_size=batchSize,shuffle=True)

/usr/local/lib/python3.6/dist-packages/torchvision/transforms/transforms.py:280: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  "please use transforms.Resize instead.")


In [ ]:
def weights_init(m):
    classname = m.__class__.__name__
    #print(classname)
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)
        

class Generator(nn.Module): 
    def __init__(self): 
        super(Generator, self).__init__() 
        self.main = nn.Sequential( 
            nn.ConvTranspose2d(in_channels=100, out_channels=512, kernel_size=4, stride=1, padding=0, bias = False), 
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace = True), 
            
            nn.ConvTranspose2d(512, 256, 4, 2, 1, bias = False), 
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace = True), 
# =============================================================================
#             nn.ConvTranspose2d(512, 256, 4, 1, 1, bias = False), 
#             nn.BatchNorm2d(256),
#             nn.LeakyReLU(0.2, inplace = True), 
# =============================================================================
            
            nn.ConvTranspose2d(256, 128, 4, 2, 1, bias = False), 
            nn.BatchNorm2d(128), 
            nn.LeakyReLU(0.2, inplace = True), 
# =============================================================================
#             nn.ConvTranspose2d(256, 128, 4, 1, 1, bias = False), 
#             nn.BatchNorm2d(128), 
#             nn.LeakyReLU(0.2, inplace = True), 
# =============================================================================
            
            nn.ConvTranspose2d(128, 64, 4, 2, 1, bias = False), 
            nn.BatchNorm2d(64), 
            nn.LeakyReLU(0.2, inplace = True), 
            
            nn.ConvTranspose2d(64, 32, 4, 2, 1, bias = False), 
            nn.BatchNorm2d(32), 
            nn.LeakyReLU(0.2, inplace = True), 
# =============================================================================
#             nn.ConvTranspose2d(128, 64, 4, 1, 1, bias = False), 
#             nn.BatchNorm2d(64), 
#             nn.LeakyReLU(0.2, inplace = True), 
# =============================================================================
            
# =============================================================================
#             nn.ConvTranspose2d(64, 32, 4, 2, 1, bias = False), 
#             nn.BatchNorm2d(32), 
#             nn.LeakyReLU(0.2, inplace = True), 
# =============================================================================
# =============================================================================
#             nn.ConvTranspose2d(64, 32, 4, 1, 1, bias = False), 
#             nn.BatchNorm2d(32), 
#             nn.LeakyReLU(0.2, inplace = True),              
#             
# =============================================================================
            nn.ConvTranspose2d(32, 3, 4, 2, 1, bias = False), 
            nn.Tanh() 
        )

    def forward(self, input): 
        output = self.main(input) 
        return output 

class Discriminator(nn.Module): 
    def __init__(self):
        super(Discriminator, self).__init__() 
        self.main = nn.Sequential( 
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=4, stride=2,padding=1, bias = False),
            nn.LeakyReLU(0.2, inplace = True), 
            
            nn.Conv2d(64, 128, 4, 2, 1, bias = False), 
            nn.BatchNorm2d(128), 
            nn.LeakyReLU(0.2, inplace = True),
            
            nn.Conv2d(128, 128, 4, 2, 1, bias = False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace = True),
            
            nn.Conv2d(128, 256, 4, 2, 1, bias = False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace = True),

            nn.Conv2d(256, 512, 4, 2, 1, bias = False),
            nn.BatchNorm2d(512), 
            nn.LeakyReLU(0.2, inplace = True),
            
            nn.Conv2d(512, 1, 4, 1, 0, bias = False), 
            nn.Sigmoid()
        )

    def forward(self, input): 
        output = self.main(input)
        return output.view(-1) #FLattening the result 

if reloadModels:
    gen = torch.load(modelFolder + 'generator.pt')
    gen = gen.to(device)
    disc = torch.load(modelFolder +'discriminator.pt')
    disc = disc.to(device)
else:
    gen= Generator().to(device)
    gen.apply(weights_init) 
    disc = Discriminator().to(device)
    disc.apply(weights_init)

In [ ]:
criterion = nn.BCELoss() 
optimizerD = optim.Adam(disc.parameters(), lr = 0.0002, betas = (0.5, 0.999))
optimizerG = optim.Adam(gen.parameters(), lr = 0.0005, betas = (0.5, 0.999))

In [ ]:
for epoch in range(55,nIter):
    for i, data in enumerate(dataloader, 0):
        disc.zero_grad()
        
        # 1.a Training the discriminator with a real image of the dataset
        real, _ = data
        input = Variable(real).to(device) 
        target = Variable(torch.ones(input.size()[0])).to(device) 
        output = disc(input).to(device) 
        errD_real = criterion(output, target)
        
        # 1.b Training the discriminator with a fake image generated by the generator
        noise = Variable(torch.randn(input.size()[0], 100, 1, 1)).to(device)
        fake = gen(noise).to(device) 
        target = Variable(torch.zeros(input.size()[0])).to(device)
        output = disc(fake.detach()).to(device) 
        errD_fake = criterion(output, target)
        
        # Backpropagating the total error
        errD = errD_real + errD_fake
        errD.backward()
        optimizerD.step()
        
        # 2 Training the generator
        gen.zero_grad()
        target = Variable(torch.ones(input.size()[0])).to(device) 
        output = disc(fake).to(device) 
        errG = criterion(output, target)
        errG.backward()
        optimizerG.step()        
        
        #3 Printing output and saving images and models
        print('[%d/%d][%d/%d] Loss_D: %.4f Loss_G: %.4f' % (epoch, nIter, i, len(dataloader), errD.item(), errG.item()))
        if i % 10 == 0:
            Vutil.save_image(real, '%s/real_samples.png' % resultsFolder, normalize = True)
            fake = gen(noise)
            Vutil.save_image(fake.data, '%s/fake_samples_epoch_%03d.png' % (resultsFolder, epoch), normalize = True)
            torch.save(gen,modelFolder +'generator.pt')
            torch.save(disc,modelFolder +'discriminator.pt')

Streaming output truncated to the last 5000 lines.
[152/1000][64/68] Loss_D: 0.3937 Loss_G: 6.3759
[152/1000][65/68] Loss_D: 0.0410 Loss_G: 6.9524
[152/1000][66/68] Loss_D: 0.5367 Loss_G: 1.7118
[152/1000][67/68] Loss_D: 0.6385 Loss_G: 8.1306
[153/1000][0/68] Loss_D: 0.0650 Loss_G: 6.8749
[153/1000][1/68] Loss_D: 0.3886 Loss_G: 2.7903
[153/1000][2/68] Loss_D: 0.1960 Loss_G: 4.2426
[153/1000][3/68] Loss_D: 0.0683 Loss_G: 5.0621
[153/1000][4/68] Loss_D: 0.0913 Loss_G: 5.7097
[153/1000][5/68] Loss_D: 0.5098 Loss_G: 1.7202
[153/1000][6/68] Loss_D: 0.8017 Loss_G: 8.1664
[153/1000][7/68] Loss_D: 0.2790 Loss_G: 4.6525
[153/1000][8/68] Loss_D: 0.1559 Loss_G: 3.6146
[153/1000][9/68] Loss_D: 0.1313 Loss_G: 4.9388
[153/1000][10/68] Loss_D: 0.1647 Loss_G: 4.2064
[153/1000][11/68] Loss_D: 0.2914 Loss_G: 2.9744
[153/1000][12/68] Loss_D: 0.1894 Loss_G: 5.6143
[153/1000][13/68] Loss_D: 0.0616 Loss_G: 5.8741
[153/1000][14/68] Loss_D: 0.2390 Loss_G: 3.5763
[153/1000][15/68] Loss_D: 0.1265 Loss_G: 4.3444

KeyboardInterrupt: ignored